In [9]:
#some imports
import json

from couchbase.bucket import Bucket
from couchbase.n1ql import N1QLQuery
import couchbase

In [2]:
#we connect to the beer-sample bucket change the ip address to your needs
cb = Bucket('couchbase://172.17.0.2/beer-sample')

In [3]:
#let's make a query

query = N1QLQuery("SELECT * " +\
                  "FROM `beer-sample` " +\
                  "WHERE brewery_id=$brewery_id",
                  brewery_id="21st_amendment_brewery_cafe")

for row in cb.n1ql_query(query):
    print(json.dumps(row, indent=2))

{
  "beer-sample": {
    "abv": 7.2,
    "brewery_id": "21st_amendment_brewery_cafe",
    "category": "North American Ale",
    "description": "Deep golden color. Citrus and piney hop aromas. Assertive malt backbone supporting the overwhelming bitterness. Dry hopped in the fermenter with four types of hops giving an explosive hop aroma. Many refer to this IPA as Nectar of the Gods. Judge for yourself. Now Available in Cans!",
    "ibu": 0,
    "name": "21A IPA",
    "srm": 0,
    "style": "American-Style India Pale Ale",
    "type": "beer",
    "upc": 0,
    "updated": "2010-07-22 20:00:20"
  }
}
{
  "beer-sample": {
    "abv": 5,
    "brewery_id": "21st_amendment_brewery_cafe",
    "category": "North American Ale",
    "description": "Deep black color, toasted black burnt coffee flavors and aroma. Dispensed with Nitrogen through a slow-flow faucet giving it the characteristic cascading effect, resulting in a rich dense creamy head.",
    "ibu": 0,
    "name": "563 Stout",
    "srm": 0

In [4]:
#let's have a look to a specific object

rv = cb.get('21st_amendment_brewery_cafe')
print(json.dumps(rv.value, indent=2))

{
  "name": "21st Amendment Brewery Cafe",
  "city": "San Francisco",
  "state": "California",
  "code": "94107",
  "country": "United States",
  "phone": "1-415-369-0900",
  "website": "http://www.21st-amendment.com/",
  "type": "brewery",
  "updated": "2010-10-24 13:54:07",
  "description": "The 21st Amendment Brewery offers a variety of award winning house made brews and American grilled cuisine in a comfortable loft like setting. Join us before and after Giants baseball games in our outdoor beer garden. A great location for functions and parties in our semi-private Brewers Loft. See you soon at the 21A!",
  "address": [
    "563 Second Street"
  ],
  "geo": {
    "accuracy": "ROOFTOP",
    "lat": 37.7825,
    "lon": -122.393
  }
}


In [5]:
#let's play with the metadata and N1QL

query = N1QLQuery("SELECT meta(beer).id,meta(beer).cas, meta(beer) " +\
                  "FROM `beer-sample` beer " +\
                  "WHERE name=$name",
                  name="563 Stout")

for row in cb.n1ql_query(query):
    print(json.dumps(row, indent=2))

{
  "$1": {
    "cas": 1501004091080572928,
    "flags": 33554438,
    "id": "21st_amendment_brewery_cafe-563_stout",
    "type": "json"
  },
  "cas": 1501004091080572928,
  "id": "21st_amendment_brewery_cafe-563_stout"
}


In [10]:
#let's store an object

toto={'name':'toto','description':'this is the best'}
try:
    result = cb.insert('toto', toto,format=couchbase.FMT_JSON)
    if not result.success:
        print("problem with insert")
except couchbase.exceptions.KeyExistsError:
    print("toto already exist, we have to delete it first")
    result=cb.remove('toto')
    print("toto removed")
    if not result.success:
        print("problem with deleting toto")
    result = cb.insert('toto', toto,format=couchbase.FMT_JSON)
    if not result.success:
        print("problem with insert")

#check the result
print(json.dumps(cb.get('toto').value, indent=2))

toto already exist, we have to delete it first
toto removed
{
  "name": "toto",
  "description": "this is the best"
}


In [11]:
query = N1QLQuery("SELECT * " +\
                  "FROM `beer-sample`  " +\
                  "WHERE name=$name",
                  name="toto")

for row in cb.n1ql_query(query):
    print(json.dumps(row, indent=2))

{
  "beer-sample": {
    "description": "this is the best",
    "name": "toto"
  }
}


In [12]:
query = N1QLQuery("SELECT name " +\
                  "FROM `beer-sample`  " +\
                  "WHERE abv>$abv",
                  abv=30)

for row in cb.n1ql_query(query):
    print(json.dumps(row, indent=2))

{
  "name": "Tactical Nuclear Penguin"
}
{
  "name": "Vetter 33"
}
{
  "name": "Norfolk Nog Old Dark Ale"
}
